# Recommender Systems with GNNs

**NOTE: Though runnable, this tutorial is outdated.  We have an example of [GCMC](https://github.com/dmlc/dgl/tree/master/examples/pytorch/gcmc) with the latest APIs but we currently lack a tutorial.  Please stay tuned.**

The academia and industries have witnessed various attempts and applications of applying GNNs to business applications including recommender systems, fraud detection, etc.

In this hands-on tutorial, we will see how GNNs can be used for a recommender system.

## Recommender System: Overview

Recommender systems are typically used on E-commerce or similar platforms.  It is responsible for predicting the items that are most likely to be interacted by a user, given his/her previous interaction history.

Based on whether the interactions have a *rating* that signifies how a user likes an item, we could categorize the interaction data as having either *explicit feedback* if the data has ratings, or *implicit feedback* otherwise.

There are a lot of methodologies for recommendation.  Here we will focus on *matrix-factorization-based* methods that expresses users and items as *embeddings* (i.e. vectors of same dimensionality), and the preference of a user to an item as the *dot product* between user and item embeddings.

More specifically, for each user we can assign a learnable vector $\boldsymbol{u}_i$, and for each item we can assign another learnable vector $\boldsymbol{v}_j$.  Additionally, we also learn a bias for each user $\beta_i$ and each item $\gamma_j$, representing the baseline rating of each user and item.  Preference prediction is expressed as $\hat{r}_{i,j}=\boldsymbol{u}_i^\top \boldsymbol{v}_j + \beta_i + \gamma_j$.

For explicit feedback datasets, we try to minimize the difference between prediction and ground truth $r_{i,j}$ for all user-item interactions:

$$
\min_{\boldsymbol{u},\boldsymbol{v}} \sum_{(i,j)\in \mathcal{D}} \left(\hat{r}_{i,j} - r_{i,j}\right)^2
$$

For implicit feedback datasets, the function to minimize is more complicated, so in this tutorial we will focus on explicit feedback datasets.

Other recommender system families include neighborhood-based models, factorization machines, etc.  But we won't go through them here.

## What Does a GNN Do in Recommender Systems?

GNNs most commonly appear as a another method of computing the user and item embeddings: instead of directly assigning a learnable vector as user and item embedding, we compute the embeddings with a GNN.  The graph for the GNN would be the interaction data itself, with users and items as two types of nodes and the interactions as edges connecting the nodes.  

Comparing against other approaches, the biggest advantage of GNNs in recommender systems is that it can potentially combine the interactions data with other kind of relational data.  If we have additional relational data such as social networks or knowledge graphs, we can combine those data with the interaction data to form a bigger graph, and still run the same GNN as a recommender system.  Other advantages include ability of introducing the information of neighbors to embeddings, and the ability to combine both user/item features and the neighborhood structure of users/items, etc.

In our hands-on, we will use the MovieLens-100K dataset.

Note that to simplify stuff in our tutorial, we would not be considering new users and items.

In [1]:
import pandas as pd

train_data = pd.read_csv('ua.base', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
test_data = pd.read_csv('ua.test', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
user_data = pd.read_csv('u.user', sep='|', header=None, encoding='latin1')
item_data = pd.read_csv('u.item', sep='|', header=None, encoding='latin1')

# Remove the entries with users and items not appearing in the training dataset
test_data = test_data[test_data['user_id'].isin(train_data['user_id']) &
                      test_data['item_id'].isin(train_data['item_id'])]

Here is a preview of the tables.

In [2]:
train_data.head(5)

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [3]:
user_data.head(5)

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
item_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Heterogeneous Graphs in DGL

If we treat our interaction data as a graph, we will have two types of nodes: users and items.  We call a graph that has multiple node types and edge types a *heterogeneous* graph.

With MovieLens-100K loaded, we will see how to build a heterogeneous graph from it.  First, we need to figure out what are the nodes and edges.

Here, since we have a user-item interaction dataset, we simply designate users and items as nodes, and interactions as edges.

In [5]:
import dgl
import torch

# Relabel the user IDs and item IDs to consecutive integers
train_data = train_data.astype({'user_id': 'category', 'item_id': 'category'})
test_data = test_data.astype({'user_id': 'category', 'item_id': 'category'})
# We need to keep the relabeling between training and test data consistent
test_data['user_id'].cat.set_categories(train_data['user_id'].cat.categories, inplace=True)
test_data['item_id'].cat.set_categories(train_data['item_id'].cat.categories, inplace=True)

train_user_ids = torch.LongTensor(train_data['user_id'].cat.codes.values)
train_item_ids = torch.LongTensor(train_data['item_id'].cat.codes.values)
train_ratings = torch.LongTensor(train_data['rating'].values)
test_user_ids = torch.LongTensor(test_data['user_id'].cat.codes.values)
test_item_ids = torch.LongTensor(test_data['item_id'].cat.codes.values)
test_ratings = torch.LongTensor(test_data['rating'].values)

# Build graph
graph = dgl.heterograph({
    # Heterogeneous graphs are organized as a dictionary of edges connecting two types of nodes.
    # We specify the edges of a type simply with a pair of user ID array and item ID array.
    ('user', 'watched', 'item'): (train_user_ids, train_item_ids),
    # Since DGL graphs are directional, we need an inverse relation from items to users as well.
    ('item', 'watched-by', 'user'): (train_item_ids, train_user_ids)
})

Using backend: pytorch
/home/gq/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  # This is added back by InteractiveShellApp.init_path()


A *metagraph* in DGL describes how edge types connect with different node types.  It is equivalent to the schema of a heterogeneous information network.

In [6]:
print(graph)

Graph(num_nodes={'item': 1680, 'user': 943},
      num_edges={('item', 'watched-by', 'user'): 90570, ('user', 'watched', 'item'): 90570},
      metagraph=[('item', 'user', 'watched-by'), ('user', 'item', 'watched')])


After we find which users and items appeared in training and test set, we now need to assign corresponding node features to the user and item nodes.  For simplicity, we would only consider the following features:

* Movie genres: the movie genres are expressed as 0-1 masks in the MovieLens-100K dataset.  We will take the mask as feature vector of the movie.
* User age: we will bin the user age by decades and treat the bin as a categorical variable.
* User gender: a categorical variable.
* User occupation: a categorical variable.

In [7]:
user_data[0] = user_data[0].astype('category')
user_data[0] = user_data[0].cat.set_categories(train_data['user_id'].cat.categories)
user_data = user_data.dropna(subset=[0])
user_data[0] = user_data[0].cat.codes
user_data = user_data.sort_values(0)
item_data[0] = item_data[0].astype('category')
item_data[0] = item_data[0].cat.set_categories(train_data['item_id'].cat.categories)
item_data = item_data.dropna(subset=[0])
item_data[0] = item_data[0].cat.codes
item_data = item_data.sort_values(0)

# Convert the age, gender, and occupation column to categorical
user_data[2] = user_data[2].astype('category')
user_data[3] = user_data[3].astype('category')
user_data[4] = user_data[4].astype('category')

user_age = user_data[1].values // 10
num_user_age_bins = user_age.max() + 1     # count the number of user age bins
user_gender = user_data[2].cat.codes.values
num_user_genders = len(user_data[2].cat.categories)
user_occupation = user_data[3].cat.codes.values
num_user_occupations = len(user_data[3].cat.categories)

item_genres = item_data[range(5, 24)].values
num_item_genres = item_genres.shape[1]

Manipulating node and edge features can be accomplished with the following:

In [8]:
# Assign user features
graph.nodes['user'].data['age'] = torch.LongTensor(user_age)
graph.nodes['user'].data['gender'] = torch.LongTensor(user_gender)
graph.nodes['user'].data['occupation'] = torch.LongTensor(user_occupation)
# Assign item features
graph.nodes['item'].data['genres'] = torch.FloatTensor(item_genres)
# Assign ratings
graph.edges['watched'].data['rating'] = torch.LongTensor(train_ratings)
graph.edges['watched-by'].data['rating'] = torch.LongTensor(train_ratings)

## Model on Heterogeneous Graphs

### Define Dataset to Sample Minibatches from

The first thing we need to do for GNN training is to define how a minibatch of examples look like.  For rating prediction tasks, a minibatch consists of pairs of users and items.  So we create a torch `Dataset` object that contains all training pairs of users and items as well as their ratings.

In [9]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_user_ids, train_item_ids, train_ratings)
test_dataset = TensorDataset(test_user_ids, test_item_ids, test_ratings)

### Define Minibatch & Neighbor Sampler

For an interaction we wish to compute the representation of its user and its item with a multi-layer GNN.  Therefore, as in the previous tutorial, we need to define a minibatch sampler that gets us the computation dependency of the user and item nodes for the multi-layer GNN.  At the same time, we also want to keep track of the training interactions themselves, i.e. which user is interacting which item.

In DGL, rating prediction could be achieved with the following steps:

1. Create a *pair graph* that consists of all the users and items in the dataset, but only with the interactions in the training minibatch.
2. *Compact* the graph to remove all unnecessary nodes, keeping only the nodes with some edges going in or out.  We provide a function `dgl.compact_graphs` for this.
3. Construct blocks bottom-up for computing the multi-layer output of the nodes, as in the previous tutorial.
4. Propagate messages top-down to obtain the output from the GNN.
5. Copy the output to the pair graph and compute the preference with `apply_edges()` method.

The following cell has the code of completing step 1 to step 3, which constitutes the steps of obtaining computation dependency for the interactions.

In [10]:
class MinibatchSampler(object):
    def __init__(self, graph, num_layers):
        self.graph = graph
        self.num_layers = num_layers
        
    def sample(self, batch):
        # Convert the list of user-item-rating triplets into a triplet of users, items, and ratings
        users, items, ratings = zip(*batch)
        users = torch.stack(users)
        items = torch.stack(items)
        ratings = torch.stack(ratings)
        
        # Create a pair graph (Step 1)
        pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        
        # Compact the graph (Step 2)
        pair_graph = dgl.compact_graphs(pair_graph)
        # Assign ratings to the graph
        pair_graph.edata['rating'] = ratings
        
        # Construct blocks (Step 3)
        seeds = {'user': pair_graph.nodes['user'].data[dgl.NID],
                 'item': pair_graph.nodes['item'].data[dgl.NID]}
        blocks = self.construct_blocks(seeds, (users, items))
        
        # Copy node features from original graph to the sampled block.
        # Note that for our model we only need to copy the features to the source side of the first block.
        # The node features of other blocks would be computed by our model.
        for feature_name in self.graph.nodes['user'].data.keys():
            blocks[0].srcnodes['user'].data[feature_name] = \
                self.graph.nodes['user'].data[feature_name][blocks[0].srcnodes['user'].data[dgl.NID]]
        for feature_name in self.graph.nodes['item'].data.keys():
            blocks[0].srcnodes['item'].data[feature_name] = \
                self.graph.nodes['item'].data[feature_name][blocks[0].srcnodes['item'].data[dgl.NID]]

        return pair_graph, blocks
        
    def construct_blocks(self, seeds, user_item_pairs_to_remove):
        blocks = []
        users, items = user_item_pairs_to_remove
        for i in range(self.num_layers):
            # We take all neighbors to form the sampled graph for computing the node representations on the
            # current layer.
            sampled_graph = dgl.in_subgraph(self.graph, seeds)
            # Find the sampled edge IDs for both directions
            sampled_eids = sampled_graph.edges['watched'].data[dgl.EID]
            sampled_eids_rev = sampled_graph.edges['watched-by'].data[dgl.EID]
            
            # A subtlety of rating prediction and link prediction is that, when we train on the pair of user A
            # and item 1, we don't want to actually tell the GNN that "user A has a connection to item 1".  So
            # we should remove all edges connecting the training pairs from the sampled graph.
            _, _, edges_to_remove = sampled_graph.edge_ids(users, items, etype='watched', return_uv=True)
            _, _, edges_to_remove_rev = sampled_graph.edge_ids(items, users, etype='watched-by', return_uv=True)
            
            sampled_with_edges_removed = sampled_graph
            if len(edges_to_remove) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove, 'watched')
                sampled_eids = sampled_eids[sampled_with_edges_removed.edges['watched'].data[dgl.EID]]
            if len(edges_to_remove_rev) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove_rev, 'watched-by')
                sampled_eids_rev = sampled_eids_rev[sampled_with_edges_removed.edges['watched-by'].data[dgl.EID]]
            
            # Create a block from the sampled graph.
            block = dgl.to_block(sampled_with_edges_removed, seeds)
            blocks.insert(0, block)
            seeds = {'user': block.srcnodes['user'].data[dgl.NID],
                     'item': block.srcnodes['item'].data[dgl.NID]}
            
            # Copy the ratings to the edges of the sampled block
            block.edges['watched'].data['rating'] = \
                self.graph.edges['watched'].data['rating'][sampled_eids]
            block.edges['watched-by'].data['rating'] = \
                self.graph.edges['watched-by'].data['rating'][sampled_eids_rev]
            
        return blocks

### Define Model

We choose a simplified version of Graph Convolutional Matrix Completion (GCMC) as the model.  Each node sends a message to its neighbors by computing a rating-specific projection of the node's representation.  Then each node gathers the messages and takes an average, projecting it afterwards.

More specifically, given a user node $i$ with its input representations $u_i^{l-1}$ on layer $l$, we

1. Find all neighbors, i.e. interacted items, of $i$.
2. For each item $j$, check the rating $r_{ij}$ between user $i$ and item $j$.  The message sent from item $j$ to user $i$ would be a rating-specific linear projection $m_{j\to i}^l \gets W_{r_{ij}}^l v_j^{l-1}$.  $W_{r_{ij}}^l$ is a learnable matrix.
3. The user $i$ aggregates all incoming messages by summing and updates itself: $u_i^l \gets \mathrm{ReLU}(W^l[\sum(m_{j\to i}^l); u_i^{l-1}])$.  $W^l$ is again a learnable matrix.

We can similarly compute $v_j$, the representation of an item node $j$.

For an implementation faithful to the paper, please see our model examples.

#### Implementation in DGL

Here we are implementing a *heterogeneous graph convolution*.  It can be decomposed into the following steps:

1. Implement or specify the convolution along each edge type individually.
2. Use `dgl.nn.HeteroGraphConv` to perform convolution on each edge type individually and aggregate the result of each node type along its incoming edge types.

The following is an implementation of GCMC convolution on a single edge type.

In [11]:
from torch import nn
import torch.nn.functional as F
import dgl.function as fn
import dgl.nn as dglnn

class GCMCConv(nn.Module):
    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        # The ratings are ranged from 1 to num_ratings, so I add 1 to the number of parameters.
        self.W_r = nn.Parameter(torch.randn(num_ratings + 1, hidden_dims, hidden_dims))
        self.W = nn.Linear(hidden_dims * 2, hidden_dims)
        
    def compute_message(self, W, edges):
        W_r = W[edges.data['rating']]
        h = edges.src['h']
        m = (W_r @ h.unsqueeze(-1)).squeeze(2)
        return m
    
    def forward(self, graph, node_features):
        with graph.local_scope():
            src_features, dst_features = node_features
            graph.srcdata['h'] = src_features
            graph.dstdata['h'] = dst_features
            # Compute messages
            graph.apply_edges(lambda edges: {'m': self.compute_message(self.W_r, edges)})
            # Aggregate messages
            graph.update_all(fn.copy_e('m', 'm'), fn.mean('m', 'h_neigh'))
            # Updates the representations of output users and items
            result = F.relu(self.W(torch.cat([graph.dstdata['h'], graph.dstdata['h_neigh']], 1)))
            return result

This object performs a heterogeneous graph convolution on both the user-watched-item relation and item-watched-by-user relation.

In [12]:
class GCMCLayer(nn.Module):
    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        self.heteroconv = dglnn.HeteroGraphConv(
            # NN Module for message passing on each individual edge type
            {'watched': GCMCConv(hidden_dims, num_ratings), 'watched-by': GCMCConv(hidden_dims, num_ratings)},
            # Aggregation strategy on each node type along incoming edge types.
            aggregate='sum')
        
    def forward(self, block, input_user_features, input_item_features):
        with block.local_scope():
            # Start from the embedding of users and items...
            h_user = input_user_features
            h_item = input_item_features
            
            src_features = {'user': h_user, 'item': h_item}
            # First copy the features from the source side to the destination side.
            # The first few nodes of the source node side would be identical to that of the destination
            # side, so we can just do the following:
            dst_features = {'user': h_user[:block.number_of_dst_nodes('user')], 'item': h_item[:block.number_of_dst_nodes('item')]}
            
            # HeteroGraphConv essentially performs a heterogeneous graph convolution by:
            # (1) Performing message passing on each edge type individually.  In this case,
            #     * Items receives messages from users along "watched" relation.
            #     * Users receives messages from items along "watched-by" relation.
            # (2) Aggregating the result on each node type along incoming edge types.  In this case,
            #     * Nothing happens since both users and items only have one incoming edge type.
            result = self.heteroconv(block, (src_features, dst_features))
            return result['user'], result['item']

Finally this object handles both the message passing and score computation.

In [13]:
class GCMCRating(nn.Module):
    def __init__(self, num_users, num_items, hidden_dims, num_ratings, num_layers):
        super().__init__()
        
        # Node-specific learnable embeddings
        self.user_embeddings = nn.Embedding(num_users, hidden_dims)
        self.item_embeddings = nn.Embedding(num_items, hidden_dims)
        
        # Transformation modules for input features of users and items
        self.U_age = nn.Embedding(num_user_age_bins, hidden_dims)
        self.U_gender = nn.Embedding(num_user_genders, hidden_dims)
        self.U_occupation = nn.Embedding(num_user_occupations, hidden_dims)
        self.U_genres = nn.Linear(num_item_genres, hidden_dims)
        
        self.layers = nn.ModuleList([
            GCMCLayer(hidden_dims, num_ratings) for _ in range(num_layers)])
        
        self.W = nn.Linear(hidden_dims, hidden_dims)
        self.V = nn.Linear(hidden_dims, hidden_dims)
        
    def forward(self, blocks):
        # Propagate messages top-down (Step 4)
        # We start with a learnable embedding for each user and item...
        user_embeddings = self.user_embeddings(blocks[0].srcnodes['user'].data[dgl.NID])
        item_embeddings = self.item_embeddings(blocks[0].srcnodes['item'].data[dgl.NID])
        
        # And add the transformation from corresponding user and item features..
        user_embeddings = user_embeddings + self.U_age(blocks[0].srcnodes['user'].data['age'])
        user_embeddings = user_embeddings + self.U_gender(blocks[0].srcnodes['user'].data['gender'])
        user_embeddings = user_embeddings + self.U_occupation(blocks[0].srcnodes['user'].data['occupation'])
        item_embeddings = item_embeddings + self.U_genres(blocks[0].srcnodes['item'].data['genres'])
        
        # Then perform a heterogeneous GCMC convolution
        for block, layer in zip(blocks, self.layers):
            user_embeddings, item_embeddings = layer(block, user_embeddings, item_embeddings)
        
        # Compute predicted preference (Step 5)
        user_embeddings = self.W(user_embeddings)
        item_embeddings = self.V(item_embeddings)
        
        return user_embeddings, item_embeddings
        
    def compute_score(self, pair_graph, user_embeddings, item_embeddings):
        with pair_graph.local_scope():
            pair_graph.nodes['user'].data['h'] = user_embeddings
            pair_graph.nodes['item'].data['h'] = item_embeddings
            pair_graph.apply_edges(fn.u_dot_v('h', 'h', 'r'))
            
            return pair_graph.edata['r']

### Define Evaluation Metric

In this tutorial we use root of mean squared error (RMSE) as the metric.

In [14]:
def rmse(pred, label):
    return ((pred - label) ** 2).mean().sqrt()

### Training Loop

Note: in this tutorial we are directly validating on the test set for simplicity.  In practice we should split out some data points in the training set to form a validation set.

In [15]:
import tqdm

# In this tutorial we consider 1-layer GNNs.
NUM_LAYERS = 1
BATCH_SIZE = 500
NUM_EPOCHS = 50
HIDDEN_DIMS = 8

sampler = MinibatchSampler(graph, NUM_LAYERS)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=False)

model = GCMCRating(graph.number_of_nodes('user'), graph.number_of_nodes('item'), HIDDEN_DIMS, 5, NUM_LAYERS)
opt = torch.optim.Adam(model.parameters())

for _ in range(NUM_EPOCHS):
    model.train()
    with tqdm.tqdm(train_dataloader) as t:
        for pair_graph, blocks in t:
            user_emb, item_emb = model(blocks)
            prediction = model.compute_score(pair_graph, user_emb, item_emb)
            loss = ((prediction - pair_graph.edata['rating']) ** 2).mean()
            opt.zero_grad()
            loss.backward()
            opt.step()
            t.set_postfix({'loss': '%.4f' % loss.item()}, refresh=False)
    model.eval()
    with tqdm.tqdm(test_dataloader) as t:
        with torch.no_grad():
            predictions = []
            ratings = []
            for pair_graph, blocks in t:
                user_emb, item_emb = model(blocks)
                prediction = model.compute_score(pair_graph, user_emb, item_emb)
                predictions.append(prediction)
                ratings.append(pair_graph.edata['rating'])

            predictions = torch.cat(predictions, 0)
            ratings = torch.cat(ratings, 0)
    print('RMSE:', rmse(predictions, ratings).item())

  1%|          | 1/182 [00:00<00:29,  6.24it/s, loss=1.5273]

RMSE: 1.271043062210083


 55%|█████▌    | 101/182 [00:12<00:10,  7.64it/s, loss=1.5048]


KeyboardInterrupt: 

### (Optional) Defining Minibatch Sampler for Implicit Feedback

For implicit feedback datasets, instead of predicting the rating between a user and an item, we often need to predict whether a user would interact the item.  Since there is no rating for us to compute the loss, we instead rely on negative sampling for training.

An example of training with negative sampling goes as follows.  For each user-item pair $(i,j)$ that appeared as an interaction (called *positive* example), we corrupt the item with a randomly-sampled item $j'$ (called *negative* example).  The model is then responsible for distinguishing whether the user-item pair is positive or negative.  The loss function for training implicit feedback can typically be any binary classification loss, for instance, a hinge loss:

$$
\mathcal{L} = \sum_{(i,j)\in D, j'} \max(\hat{r}_{i,j'} - \hat{r}_{i,j} + 1, 0)
$$

In graph learning we usually refer to this problem as *link prediction*, i.e. predicting whether an edge exists between two nodes.  In DGL it could be achieved with the following steps (the difference from rating prediction highlighted in bold):

1. **Create two *pair graphs* that consists of all the users and items in the dataset, but only with the interactions in the training minibatch of positive examples and negative examples respectively.**
2. *Compact* **both graphs** to remove all unnecessary nodes, keeping only the nodes with some edges going in or out.
3. Construct blocks bottom-up for computing the multi-layer output of the nodes, as in the previous tutorial.
4. Propagate messages top-down to obtain the output from the GNN.
5. Copy the output to **both pair graphs** and compute the preference with `apply_edges()` method.

In [ ]:
class LinkPredictionMinibatchSampler(MinibatchSampler):
    def __init__(self, graph, num_layers):
        self.graph = graph
        self.num_layers = num_layers
        
    def sample(self, batch):
        # Convert the list of user-item-rating triplets into a pairs of users and items
        users, items, _ = zip(*batch)
        users = torch.stack(users)
        items = torch.stack(items)
        # Get corrupted items for negative examples
        neg_items = torch.randint(0, self.graph.number_of_nodes('item'), (len(users),))
        
        # Create a pair graph for positive examples and negative examples (Step 1)
        pos_pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        neg_pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, neg_items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        
        # Compact the graph (Step 2)
        pos_pair_graph, neg_pair_graph = dgl.compact_graphs([pos_pair_graph, neg_pair_graph])
        
        # Construct blocks (Step 3)
        # Note that pos_pair_graph and neg_pair_graph have the same set of users and items, so we only need
        # to check one of them to get the seed nodes.
        seeds = {'user': pos_pair_graph.nodes['user'].data[dgl.NID],
                 'item': pos_pair_graph.nodes['item'].data[dgl.NID]}
        # Note that here we would also remove edges connecting between users and both the corresponding positive
        # and negative items appearing in the minibatch.
        blocks = self.construct_blocks(seeds, (torch.cat([users, users]), torch.cat([items, neg_items])))
        
        # Copy node features from original graph to the sampled block.
        # Note that for our model we only need to copy the features to the source side of the first block.
        for feature_name in self.graph.nodes['user'].data.keys():
            blocks[0].srcnodes['user'].data[feature_name] = \
                self.graph.nodes['user'].data[feature_name][blocks[0].srcnodes['user'].data[dgl.NID]]
        for feature_name in self.graph.nodes['item'].data.keys():
            blocks[0].srcnodes['item'].data[feature_name] = \
                self.graph.nodes['item'].data[feature_name][blocks[0].srcnodes['item'].data[dgl.NID]]
            
        return pos_pair_graph, neg_pair_graph, blocks

With minibatch sampler defined for link prediction, we only need to rewrite the training loop.